This notebook should be revised because some stations appear many times. Is it because we remove the firt "0"?

# different types of graphs 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm.auto import tqdm
from sklearn.linear_model import LinearRegression

In [ ]:
# importing necessary functions from other files

import os
import sys

# to facilitate the use of notebooks
%load_ext autoreload
%autoreload 2

# Get the current directory of the notebook
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
functions_dir = os.path.join(parent_dir, "common")
print(functions_dir)

# Add this directory to sys.path
if functions_dir not in sys.path:
    sys.path.append(functions_dir)

from merge_metadata import add_HS, add_coord

## MF stations

In [ ]:
# Get the current directory of the notebook
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir, os.pardir))
# Import phychem data
param_file = os.path.join(parent_dir, "phychem_data", "stations", "mostfreq_sta.txt")

In [ ]:
# Reading the list from the file
mostfreq_sta = []
with open(param_file, 'r') as file:
    for line in file:
        # Convert each line to the appropriate type
        mostfreq_sta.append(str(line.strip()))

len(mostfreq_sta)

## Excel files

In [ ]:
# read excel files

xlsx_names = os.listdir(os.path.join(os.getcwd(),"phychem_param_80_19", "param_station_xlsx"))
# get stations ID from name of file
list_station_ID = []
# store df to create a final one
list_df_xlsx = []

for xlsx_name in tqdm(xlsx_names):

    station_ID = xlsx_name.split("_")[1].split(".")[0]
    
    if station_ID.startswith("0"):
        station_ID = station_ID.split("0",1)[1]

    if station_ID in list_station_ID:
        continue
        # print(f"station was already added: {station_ID}")
    else:
        list_station_ID.append(station_ID)
        df = pd.read_excel(os.path.join(os.getcwd(),"phychem_param_80_19","param_station_xlsx", xlsx_name))
        list_df_xlsx.append(df)

In [ ]:
# remove 0 in first position in station ID

for i,ID in enumerate(list_station_ID):
    if ID.startswith('0'):
        ID = ID.lstrip('0')
        list_station_ID.pop(i)
        list_station_ID.insert(i, ID)

In [ ]:
# store concentration values in one df per station

name_cols = ["station_ID"] # already in english
years = []
seasons = []

concentration_df = pd.DataFrame()

for index, df_xlsx in tqdm(enumerate(list_df_xlsx),total=len(list_df_xlsx)): 

    if index==0: # get only first time
        years += df_xlsx["Year"].tolist()
        seasons += df_xlsx["Season"].tolist()
        name_cols += df_xlsx.columns[2:].tolist()
        concentration_df = pd.DataFrame(columns=name_cols)

    df2 = pd.DataFrame(columns=name_cols[1::])
    df2.loc[0] = [[] for i in range(len(name_cols[1::]))]

    med_per_col = []
    for col in name_cols[1::]: # neglect first col of station_ID)
        df2.at[0,col] = df_xlsx[col].tolist()

    
    df2.insert(loc=0, column="station_ID", value=list_station_ID[index])

    concentration_df = pd.concat([concentration_df, df2], ignore_index=True)
    


In [ ]:
# store all index corresponding to each season. 
seas_indx = {seas: np.where(np.array(seasons)==seas)[0].tolist() for seas in np.unique(seasons)}

In [ ]:
# MF stations
freq_conc_df = concentration_df[concentration_df["station_ID"].isin(mostfreq_sta)].reset_index(drop=True)
freq_conc_df.shape, freq_conc_df["station_ID"].unique().shape

## delete extreme values

In [ ]:
# Function to del outliers from carbon values greater than 100 with np.nan
def replace_outliers(value_list):
    return [np.nan if (isinstance(x, (int, float)) and x > 100) else x for x in value_list]
    
concentration_df["Organic carbon"] = concentration_df["Organic carbon"].apply(replace_outliers)
freq_conc_df["Organic carbon"] = freq_conc_df["Organic carbon"].apply(replace_outliers)

## add HS

In [ ]:
concentration_df = add_HS(type="Physico-chemical", left_df = concentration_df, left_id="station_ID")
freq_conc_df = add_HS(type="Physico-chemical", left_df = freq_conc_df, left_id="station_ID")

## calculate median

In [ ]:
# to get median with no nan
def grouping_df(df, name_cols=name_cols):
   
    # transform -1 into nan
    for col in name_cols[1:]:
        df[col] = df[col].apply(lambda x : [np.nan if value == -1 else value for value in x ])

    # create functions stored in a dict
    values = [list] + [lambda x: np.nanmedian(np.stack(x, axis=0), axis=0) for i in range(len(name_cols[1:]))]
    hs_dict = {key: value for key, value in zip(name_cols, values)}

    # apply functions during groupby
    hs_grouped_df = df.groupby("ORD_STRA").agg(hs_dict).reset_index()

    return hs_grouped_df

In [ ]:
hs_grouped_conc, hs_grouped_freqconc = grouping_df(concentration_df), grouping_df(freq_conc_df)

# graph

methode pour x ticks:
recupreer pour chaque saison, premiere annee non nan
derniere annee non nan
faire un arange entre les deux

In [ ]:
def graph_conc(hs_grouped_df, stations="All stations", name_cols=name_cols, seas="all", seas_indx=seas_indx, years=years):

    fig_row = len(name_cols[1:])//2
    fig, ax = plt.subplots(fig_row, 2, figsize=(20,15), layout="tight")

    if seas!="all":
        index = seas_indx[seas]
        years = np.array(years)[index] # take corresponding years (manage 1980 and 2019)
            

    for i, col_df in enumerate(name_cols[1:]):
        row = i % fig_row
        col = i // fig_row

        for j, row_df in hs_grouped_df.iterrows():

            if seas!="all":
                value = row_df[col_df][index]   
            else:
                value = row_df[col_df]
                
            ax[row][col].plot([i for i in range(len(value))], value, "-o", markersize=2, label=f"HS {j+1}")
   

        # xticks = ax[row][col].get_xticks()
        # ax[row][col].set_xticks(xticks[1:-1], labels = [years[math.ceil(k)] for k in xticks[1:-1]]) # modify to accept first element xticks
        ax[row][col].legend()
        ax[row][col].set_title(f"{col_df}", fontsize=18)

    if seas!="all": title = f"{stations} - Concentrations {seas} season 1980-2019"
    else: title = f"{stations} - Concentrations all seasons 1980-2019"
    
    fig.suptitle(title, fontsize=25)



In [ ]:
# all stations
graph_conc(hs_grouped_df=hs_grouped_conc, seas="WT")

In [ ]:
# most frequent stations
graph_conc(hs_grouped_df=hs_grouped_freqconc, stations="Most frequent stations", seas="WT")

# Find best trend

### This part should be revised

                        slope1 error1 #nan1 slope2 error2 #nan2
1980
1981
80-82 ; 83-19


In [ ]:
index = seas_indx["SM"]

values = hs_grouped_conc.at[5, "Dissolved oxygen"]
SM_val = np.array(values[index])
year_unique = np.unique(years).tolist()

In [ ]:
def calculate_seg_error(data, split_index):
    # Diviser les données en deux segments
    segment1 = data[:split_index+1] # split index included
    segment2 = data[split_index:] # split index included

    X1 = np.where(~np.isnan(segment1))[0] # get index of those who are not nan
    X2 = np.where(~np.isnan(segment2))[0]

    nb_nan1 = len(segment1) - len(X1) # get number of nan
    nb_nan2 = len(segment2) - len(X2)

    # remove nan
    segment1 = segment1[~np.isnan(segment1)] # get data without nan
    segment2 = segment2[~np.isnan(segment2)]

    # Ajuster une régression linéaire pour chaque segment
    model1 = LinearRegression()
    model2 = LinearRegression()
    
    model1.fit(X1.reshape(-1, 1), segment1.reshape(-1, 1))
    model2.fit(X2.reshape(-1, 1), segment2.reshape(-1, 1))
    
    # Prédire les valeurs
    predictions1 = model1.predict(X1.reshape(-1, 1))
    predictions2 = model2.predict(X2.reshape(-1, 1))
    
    # Calculer la somme des erreurs au carré pour chaque segment
    error1 = np.mean(np.abs(predictions1 - segment1.reshape(-1, 1)))
    error2 = np.mean(np.abs(predictions2 - segment2.reshape(-1, 1)))

    slope1 = model1.coef_[0]
    slope2 = model2.coef_[0]

    
    return [slope1, error1, nb_nan1, slope2, error2, nb_nan2]


In [ ]:
def perform_segmentation(value=SM_val, years=year_unique):

    df = pd.DataFrame(columns= ["seg_year", "slope1", "err1", "nb_nan1", "slope2", "err2", "nb_nan2"])

    for i, seg_year in enumerate(years[1:-1]): # traiter apres
        
        df2 = pd.DataFrame(columns= ["seg_year", "slope1", "err1", "nb_nan1", "slope2", "err2", "nb_nan2"]
                           )
        
        # if i==0:
        #     calculate_seg_error(value, i)

        if np.isnan(value[i]):
            result = [seg_year] + [np.nan for i in range(6)]

        else:
            result = [seg_year] + calculate_seg_error(value, i+1) # +1 changer apres
            
        print(result, i)

        for j, col in enumerate(df2.columns):
            df2[col] = result[j]

        df = pd.concat((df, df2), ignore_index=True)

    return df


In [ ]:
perform_segmentation()

In [ ]:
def find_best_split(data):
    min_error = float('inf')
    best_split = None
    
    for i in range(1, len(data) - 1):
        current_error = calculate_total_error(data, i)
        if current_error < min_error:
            min_error = current_error
            best_split = i
    
    return best_split

best_split_index = find_best_split(data)


In [ ]:
# Diviser les données en deux segments
segment1 = data[:best_split_index]
segment2 = data[best_split_index:]

# Ajuster une régression linéaire pour chaque segment
model1 = LinearRegression()
model2 = LinearRegression()

X1 = np.array(range(len(segment1))).reshape(-1, 1)
X2 = np.array(range(len(segment2))).reshape(-1, 1)

model1.fit(X1, segment1['concentration'])
model2.fit(X2, segment2['concentration'])

# Prédire les valeurs
predictions1 = model1.predict(X1)
predictions2 = model2.predict(X2)

# Tracer les résultats
plt.figure(figsize=(12, 6))
plt.plot(data.index, data['concentration'], label='Data')
plt.plot(segment1.index, predictions1, label='Segment 1 Trend', color='r')
plt.plot(segment2.index, predictions2, label='Segment 2 Trend', color='g')
plt.axvline(x=data.index[best_split_index], color='k', linestyle='--', label='Change Point')
plt.xlabel('Date')
plt.ylabel('Concentration de Nitrogen')
plt.legend()
plt.show()
